In [1]:
import pandas as pd
import scipy
from scipy import sparse
import numpy as np
from sklearn.model_selection import train_test_split
import pysparnn.cluster_index as ci

#df_ratings = pd.read_csv('ml-20m/ratings.csv', skiprows=[0], names=["user_id", "movie_id", "rating", "timestamp"]).drop(columns=['timestamp']).head(1000000)
df_ratings = pd.read_csv('ml-1m/ratings.dat', names=["user_id", "movie_id", "rating", "timestamp"],
            header=None, sep='::', engine='python')
matrix_df = df_ratings.pivot(index='movie_id', columns='user_id', values='rating').fillna(0).astype(bool).astype(int)

#idx to id and reverse dicts
c=0
hashmap = {} 
reverse_hashmap = {}
for i in matrix_df.index.tolist():
    hashmap[c] = i
    reverse_hashmap[i] = c
    c+=1

#get first nonzero elements for validation
validation_movies = matrix_df.ne(0).idxmax()

#make validation movies unrated
for index, row in validation_movies.items():
    matrix_df[index][row] = 0

#get second nonzero elements for validation
validation_movies2 = matrix_df.ne(0).idxmax()

#make validation movies rated
for index, row in validation_movies.items():
    matrix_df[index][row] = 1

#make validation movies2 unrated
for index, row in validation_movies2.items():
    matrix_df[index][row] = 0
    
um_matrix = scipy.sparse.csr_matrix(matrix_df.values)

In [2]:
#list of movies that each user rated
user_hists = []
for user in matrix_df:
    a = [i for i, e in enumerate(matrix_df[user].tolist()) if e != 0]
    user_hists.append(a)

In [3]:
doc_index = np.array(range(len(matrix_df)))

snn = ci.MultiClusterIndex(um_matrix, doc_index, num_indexes=10)

results = snn.search(um_matrix, k=len(matrix_df), return_distance=True, num_indexes=10, k_clusters=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Version

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysparnn/matrix_distance.py:192: RuntimeWarning: divide by zero encountered in true_divide
  magnitude = 1.0 / (a_root_sum_square * self.matrix_root_sum_square)
/Library/Frameworks/Python.framework/Version

In [4]:
results_dic=[]
for i in results:
    results_dic.append(dict((int(y), x) for x, y in i))

In [47]:
#for each user get average distance of the movies that user rated to retrieve top k movies to recommend
avg_dict = {}
for index, user in enumerate(user_hists):
    user_dict={}    
    for movie in user:
        for m in results_dic[movie]:
            if m not in user_dict:
                user_dict[m]=results_dic[movie][m]
            else:
                user_dict[m]+=results_dic[movie][m]
    avg_dict[index]=dict(sorted(user_dict.items(), key=lambda x: x[1]))
    for m in user_hists[index]:
        del avg_dict[index][m]
    avg_dict[index] = {k: avg_dict[index][k] for k in list(avg_dict[index])[:10]}

In [48]:
recall = 0
for user in avg_dict:
    if reverse_hashmap[validation_movies2[user+1]] in avg_dict[user]: 
        recall+=1
print(recall)
recall=recall/len(user_hists)

657


In [49]:
#ndcg calculation
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

results=[]
for key in avg_dict:
    results.append(list(avg_dict[key].keys()))   

In [50]:
#mrp calculation
def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

for i in range(0, len(results)):
    for x in range(0,len(results[i])):
        if results[i][x]==reverse_hashmap[validation_movies2[i+1]]:
            results[i][x]=1
        else:
            results[i][x]=0

In [51]:
mrr = mean_reciprocal_rank(results)
ndcg=0
for i in results:
    ndcg+=ndcg_at_k(i, 10)
ndcg=ndcg/len(results)

In [52]:
print(recall)
print(mrr)
print(ndcg)

0.10877483443708609
0.04489047881845896
0.0691746069526115
